In [4]:
from transformers import *

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
# from official.nlp import optimization  # to create AdamW optimizer

# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences

# import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [7]:
data = pd.read_csv('drive/MyDrive/bert2/big_class/data3.csv')
data

,UT,ABSTRACT,DISC,text,target,label
0,369459800018,"Real-time, nondestructive estimation of crop n...",A2,The optimum vegetation indices (VIs) obtained ...,A,9
1,383001900005,To promote the stress resilience of sweet pepp...,A2,This study aimed to determine the changes in t...,A,9
2,381837100014,Soil degradation has seriously threatened glob...,A2,"Therefore, the individual or combined effects ...",A,9
3,369517800007,We measured parameters of photosynthetic capac...,A2,We measured parameters of photosynthetic capac...,A,9
4,382969900003,The transformation of humus substances resulti...,A2,The transformation of humus substances resulti...,A,9
...,...,...,...,...,...,...
61615,382152500011,We evaluated 160 hip joint radiographs of 40 d...,Z6,The aims were to evaluate the presence of the ...,Z,8
61616,366987700001,Background: Enterotoxigenic Escherichia coli (...,Z6,This study aimed to determine the VF and antim...,Z,8
61617,375743900005,Large animal veterinary practice is more and m...,Z6,The results of this study suggest that it is i...,Z,8
61618,375336400003,Rhodococcus equi is the causative agent of rho...,Z6,equi detection in clinical specimens by the de...,Z,8


In [8]:
df = data.copy()

In [9]:
df[df['text'].isna()]

,UT,ABSTRACT,DISC,text,target,label
14,383137300004,"In pot culture, we studied the effects of whea...",A2,NaN,A,9
101,375210700017,Species richness and turnover rates differed b...,A2,NaN,A,9
130,373930500026,Leaf nutrient content indicate the nutritional...,A2,NaN,A,9
225,368299700009,We developed and tested spectroscopic models d...,A2,NaN,A,9
248,375078700024,Studies have been performed to identify the pr...,A2,NaN,A,9
...,...,...,...,...,...,...
61553,376810700031,To evaluate the prognostic criteria for identi...,Z6,NaN,Z,8
61564,374230800006,The SV40 viral oncogene has been used since th...,Z6,NaN,Z,8
61569,366437800008,Five calves were inoculated orally at 2 weeks ...,Z6,NaN,Z,8
61604,366437200004,Patchy meningeal and parenchymal contrast enha...,Z6,NaN,Z,8


In [10]:
df['text'] = df['text'].fillna(df['ABSTRACT'])

In [11]:
df[df['text'].isna()]

,UT,ABSTRACT,DISC,text,target,label


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( df['text'], df['label'], test_size=0.2, random_state=42)
X_train

57213    This study explored the contextual relationshi...
34935    This will act as a guide for local policy make...
48117    The preparation and spectral study of one copp...
35348    Harry Truman in this book epitomises 1946 Amer...
40899    OBJECTIVE To evaluate the perceived current ne...
                               ...                        
54343    Findings - Using the novel graph database appr...
38158    HGF cells were exposed with NTAPPJ for 1, 2, a...
860      To determine the effect of CA on generalist ar...
15795    The bearing performance characteristics in ter...
56422    The article demonstrates that while some moder...
Name: text, Length: 49296, dtype: object

In [13]:
!wget https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/tensorflow_models/scibert_scivocab_uncased.tar.gz
!tar -xvf ./scibert_scivocab_uncased.tar.gz

--2022-08-06 04:19:50--  https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/tensorflow_models/scibert_scivocab_uncased.tar.gz
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.92.194.104
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.92.194.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1216161420 (1.1G) [application/x-tar]
Saving to: ‘scibert_scivocab_uncased.tar.gz’

scibert_scivocab_un 100%[===================>]   1.13G  21.7MB/s    in 54s     

2022-08-06 04:20:45 (21.4 MB/s) - ‘scibert_scivocab_uncased.tar.gz’ saved [1216161420/1216161420]

scibert_scivocab_uncased/
scibert_scivocab_uncased/bert_model.ckpt.data-00000-of-00001
scibert_scivocab_uncased/bert_model.ckpt.index
scibert_scivocab_uncased/vocab.txt
scibert_scivocab_uncased/bert_model.ckpt.meta
scibert_scivocab_uncased/bert_config.json


In [14]:
import os
os.environ["WANDB_API_KEY"] = "0" ## to silence warning
!transformers-cli convert --model_type bert \
  --tf_checkpoint './scibert_scivocab_uncased/bert_model.ckpt' \
  --config './scibert_scivocab_uncased/bert_config.json' \
  --pytorch_dump_output './scibert_scivocab_uncased/pytorch_model.bin'

Building PyTorch model from configuration: BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "type_vocab_size": 2,
  "vocab_size": 31090
}

Converting TensorFlow checkpoint from /content/scibert_scivocab_uncased/bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [768]
Loading TF weight bert/embeddings/LayerNorm/beta/adam_m with shape [768]
Loading TF weight bert/embeddings/LayerNorm/beta/adam_v with shape [768]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [768]
Loading TF weight bert/embeddings/LayerNorm/gamma/adam_m with shape [768]
Loading TF weight bert/embeddings/LayerNorm

In [15]:
# from transformers import *

bert_model_name = './scibert_scivocab_uncased'

config = BertConfig.from_json_file('./scibert_scivocab_uncased/bert_config.json')

In [16]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)

In [17]:
model = TFBertModel.from_pretrained(bert_model_name, from_pt=True, config = config)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [18]:
from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 128

def tokenize_sentences(sentences, tokenizer, max_seq_len = 128):
    tokenized_sentences = []

    for sentence in sentences:
        tokenized_sentence = tokenizer.encode(
                            sentence,                  # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_seq_len,  # Truncate all sentences.
                    )
        
        tokenized_sentences.append(tokenized_sentence)

    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        attention_masks.append(att_mask)

    return np.asarray(attention_masks)



In [19]:
train_input_ids = tokenize_sentences(X_train, tokenizer, MAX_LEN)
train_input_ids = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
train_attention_masks = create_attention_masks(train_input_ids)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [20]:
test_input_ids = tokenize_sentences(X_test, tokenizer, MAX_LEN)
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
test_attention_masks = create_attention_masks(test_input_ids)

In [21]:
from keras.layers import LSTM

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics=tf.keras.metrics.SparseCategoricalAccuracy('accuracy')


def build_model(model_layer, learning_rate, dense_dim = 15):
    
    #define inputs
    input_ids = tf.keras.Input(shape=(128,),dtype='int64')
    attention_masks = tf.keras.Input(shape=(128,),dtype='int64')
 
    
    #insert BERT layer
    transformer_layer = model_layer([input_ids,attention_masks])
    
    #choose only last hidden-state
    output = transformer_layer[0]
    output = tf.keras.layers.SpatialDropout1D(0.2)(output)
    output = tf.keras.layers.Conv1D(64, 3, activation='relu')(output)
    output = tf.keras.layers.Bidirectional(LSTM(128))(output)
    output = tf.keras.layers.Dense(dense_dim,activation='sigmoid')(output)

    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)

    model.compile(tf.keras.optimizers.Adam(lr=learning_rate), loss=loss,metrics=metrics)
    
    return model

In [ ]:
model = TFBertModel.from_pretrained(bert_model_name, from_pt=True, config = config)

loading weights file ./scibert_scivocab_uncased/pytorch_model.bin
Loading PyTorch weights from /content/scibert_scivocab_uncased/pytorch_model.bin
PyTorch checkpoint contains 134,451,942 parameters
Loaded 109,918,464 parameters in the TF 2.0 model.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch mode

In [22]:
SciBERT= build_model(model, learning_rate = 1e-5)
SciBERT.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109918464   ['input_1[0][0]',                
                                thPooling(last_hidd               'input_2[0][0]']                
                                en_state=(None, 128                                               
                                , 768),                                                       

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


### 训练模型

In [23]:
callbacks_list = [tf.keras.callbacks.ModelCheckpoint('scibert3.h5', monitor='val_accuracy', save_best_only = True, save_weights_only = True)]

In [24]:
history = SciBERT.fit((train_input_ids, train_attention_masks),y_train,batch_size =32,                  
                epochs=3,
                validation_split = 0.25,
                callbacks=callbacks_list
                )

Epoch 1/3


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


1156/1156 [==============================] - ETA: 0s - loss: 0.9867 - accuracy: 0.7306

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


1156/1156 [==============================] - 676s 562ms/step - loss: 0.9867 - accuracy: 0.7306 - val_loss: 0.6766 - val_accuracy: 0.7951
Epoch 2/3
1156/1156 [==============================] - 645s 558ms/step - loss: 0.5673 - accuracy: 0.8306 - val_loss: 0.6079 - val_accuracy: 0.8137
Epoch 3/3
1156/1156 [==============================] - 645s 558ms/step - loss: 0.4388 - accuracy: 0.8685 - val_loss: 0.5928 - val_accuracy: 0.8181


In [26]:
from sklearn.metrics import classification_report

In [27]:
predictions = SciBERT.predict([test_input_ids,test_attention_masks])

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


In [28]:
# y_test = df_test.label.values
y_pred = np.argmax(predictions, axis=-1)

In [29]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.81      0.78       836
           1       0.93      0.86      0.90       779
           2       0.78      0.83      0.81       847
           3       0.85      0.86      0.86       843
           4       0.91      0.86      0.88       853
           5       0.71      0.62      0.66       805
           6       0.85      0.90      0.87       823
           7       0.79      0.90      0.84       823
           8       0.80      0.80      0.80       823
           9       0.79      0.80      0.80       824
          10       0.86      0.91      0.89       815
          11       0.81      0.77      0.79       832
          12       0.72      0.70      0.71       833
          13       0.78      0.69      0.73       798
          14       0.91      0.92      0.91       790

    accuracy                           0.82     12324
   macro avg       0.82      0.82      0.81     12324
weighted avg       0.82   

In [30]:
bert_report = pd.DataFrame(classification_report(y_test, y_pred,output_dict=True)).T
bert_report.to_csv('drive/MyDrive/bert2/big_class/report_scibert2.csv')